# Um guia de como testar as funções do anansi

## "Brincando" com os modelos nos bancos de dados

In [1]:
from anansi.tradingbot.models import *
from anansi.tradingbot import traders
from pony.orm import *

In [ ]:
# Adicionando usuário
my_user_first_name = "Marcus"

create_user(first_name=my_user_first_name, 
                   last_name="Silva", 
                   email = "{}@email.com".format(my_user_first_name.lower()))

In [ ]:
# Lendo todos os usuários

users = select(user for user in User)
users.show()

In [ ]:
# Criando uma operação padrão

my_user = User.get(first_name="Marcus")
create_operation(user=my_user)

## Instanciando um trader

In [2]:
my_op = Operation.get(id=1)
my_trader = traders.DefaultTrader(operation=my_op)

In [3]:
my_klines = my_trader.klines.newest(2135)

In [4]:
my_klines

,Open_time,Open,High,Low,Close,Volume
0,2017-10-10 12:00:00,4790.01,4960.00,4680.59,4783.06,688.531928
1,2017-10-11 00:00:00,4783.06,4852.22,4710.00,4818.99,612.082536
2,2017-10-11 12:00:00,4804.93,4881.61,4751.01,4821.43,141.346860
3,2017-10-12 00:00:00,4821.43,5265.50,4810.16,5184.00,540.763389
4,2017-10-12 12:00:00,5184.00,5439.99,5173.10,5430.00,735.938093
...,...,...,...,...,...,...
2130,2020-09-09 12:00:00,10221.47,10343.00,10165.99,10219.20,26676.590083
2131,2020-09-10 00:00:00,10219.29,10418.00,10070.83,10292.62,26599.923830
2132,2020-09-10 12:00:00,10292.61,10483.35,10250.02,10336.87,31653.829920
2133,2020-09-11 00:00:00,10336.86,10380.34,10200.00,10252.60,23510.534282


In [ ]:
my_op.stop_loss_name

In [ ]:
operations = select(operation for operation in Operation)

In [ ]:
operations.show()

In [ ]:
my_user = User.get(first_name="Daniel_segundo")

In [ ]:
my_user.id

In [ ]:
my_user.update_first_name_to("carlos_SEGUNDO")

In [ ]:
my_user.first_name

In [ ]:
stat

In [ ]:
my_operation = Operation._

In [ ]:
jose2 = models.User.add(first_name="Jose2", last_name="Silva", email = "joao@email.com")

In [ ]:
type(jose2)

In [ ]:
My_trader = models.Operation.

In [ ]:
type(My_trader)

In [ ]:
# Criando uma operação

my_operation = models.create_an_operation(user=user)

In [ ]:
user.id

In [ ]:
user.to_dict()

In [ ]:
user.user_name

In [ ]:

MyTrader = models.Trader(costumer=user, position=models.Position(), exchange = "binance", symbol = "BTCUSDT")

## Klines tratadas e prontas para uso - com indicadores inclusive - utilizando o módulo "*klines*"

In [ ]:
from anansi.marketdata import klines

In [ ]:
BinanceKlines = klines.FromBroker(broker_name="binance", symbol="BTCUSDT", time_frame="1h")

In [ ]:
newest_klines = BinanceKlines.newest(2167)

### Aplicando indicador médias móveis

In [ ]:
newest_klines.apply_indicator.trend.simple_moving_average(number_of_candles=35, 
                                                          create_indicator_column=True)

In [ ]:
newest_klines

## Klines "cruas", utilizando diretamente o módulo "*data_brokers*"

In [ ]:
from anansi.marketdata import data_brokers

In [ ]:
BinanceBroker = data_brokers.BinanceDataWrapper() 

In [ ]:
my_klines = BinanceBroker.get_klines(symbol="BTCUSDT", time_frame="1m", show_only_desired_info=False)

In [ ]:
my_klines

In [ ]:
import anansi
import pendulum

In [ ]:
user.get_name()

In [ ]:
user.alter_name_to("Jonas")

In [ ]:
user.get_name()

In [ ]:
commit()

In [ ]:
    status = Required(str, default=Default.status)
    costumer = Required("Customer", columns=["name"])
    mode = Required(str, default=Default.mode)
    position = Required("Position")
    exchange = Required(str)
    symbol = Required(str)
    classifier_name = Required(str, default=Default.classifier)
    classifier_parameters = Optional(Json)
    stop_loss_name = Required(str, default=Default.stop_loss)
    stop_loss_parameters = Optional(Json)

In [ ]:
MyTrader.user_name

In [ ]:
from pony.orm import *
commit()

In [ ]:
MyTrader = traders.DefaultTrader()

In [ ]:
MyTrader.convert_time_frame("1d")

In [ ]:
MyTrader.market.symbol

In [ ]:
traders.db_handlers.SQLite3("teste", "teste").table_exists

In [ ]:
empty_ = dict()

In [ ]:
type(empty_)

In [ ]:
print("hello")

In [ ]:
my_trader = models.Trader.get(id=1)

In [ ]:
print(my_trader.position.side)